In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# !pip install py4j

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"


# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,069 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [31.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,199 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu f

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

home_sales.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales.createOrReplaceTempView('home')


In [21]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("select YEAR(date), round(avg(price),2) as avg_price from home \
              where bedrooms = 4\
              group by YEAR(date) order by YEAR(date)"\
                  ).show()


+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [24]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("select date_built, round(avg(price),2) as avg_price from home \
              where bedrooms = 3 and bathrooms = 3 \
              group by date_built order by date_built"\
                  ).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [25]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("select date_built, round(avg(price),2) as avg_price from home \
              where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living > 2000 \
              group by date_built order by date_built"\
                  ).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|297619.46|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280527.62|
+----------+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, avg(price) FROM home where price > 350000 group by view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|        avg(price)|
+----+------------------+
|  31|399856.95135135134|
|  85|1056336.7435897435|
|  65| 736679.9333333333|
|  53|          755214.8|
|  78|1080649.3666666667|
|  34| 401419.7451923077|
|  81|1053472.7878787878|
|  28|402124.62176165805|
|  76|1058802.7777777778|
|  26|401506.96774193546|
|  27| 399537.6586826347|
|  44|400598.04761904763|
|  12| 401501.3243243243|
|  91| 1137372.731707317|
|  22| 402022.6847826087|
|  93|1026006.0606060605|
|  47| 398447.4976744186|
|   1| 401044.2513368984|
|  52| 733780.2608695652|
|  13|         398917.98|
+----+------------------+
only showing top 20 rows

--- 0.811168909072876 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, avg(price) FROM home where price > 350000 group by view").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|        avg(price)|
+----+------------------+
|  31|399856.95135135134|
|  85|1056336.7435897435|
|  65| 736679.9333333333|
|  53|          755214.8|
|  78|1080649.3666666667|
|  34| 401419.7451923077|
|  81|1053472.7878787878|
|  28|402124.62176165805|
|  76|1058802.7777777778|
|  26|401506.96774193546|
|  27| 399537.6586826347|
|  44|400598.04761904763|
|  12| 401501.3243243243|
|  91| 1137372.731707317|
|  22| 402022.6847826087|
|  93|1026006.0606060605|
|  47| 398447.4976744186|
|   1| 401044.2513368984|
|  52| 733780.2608695652|
|  13|         398917.98|
+----+------------------+
only showing top 20 rows

--- 0.556110143661499 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales.write.partitionBy("date_built").mode("overwrite").parquet('home')

In [13]:
# 11. Read the parquet formatted data.
p_home=spark.read.parquet('home')

In [14]:
# 12. Create a temporary table for the parquet data.
p_home.createOrReplaceTempView('home')

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, avg(price) FROM home where price > 350000 group by view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|        avg(price)|
+----+------------------+
|  31|399856.95135135134|
|  85|1056336.7435897435|
|  65| 736679.9333333333|
|  53|          755214.8|
|  78|1080649.3666666667|
|  34| 401419.7451923077|
|  81|1053472.7878787878|
|  28|402124.62176165805|
|  76|1058802.7777777778|
|  26|401506.96774193546|
|  27| 399537.6586826347|
|  44|400598.04761904763|
|  12| 401501.3243243243|
|  91| 1137372.731707317|
|  22| 402022.6847826087|
|  93|1026006.0606060605|
|  47| 398447.4976744186|
|   1| 401044.2513368984|
|  52| 733780.2608695652|
|  13|         398917.98|
+----+------------------+
only showing top 20 rows

--- 1.122206211090088 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home')


False